In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [2]:
num_classes = 2
images_size = 224
batch_size_training = 100
batch_size_validation = 100

In [3]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_generator = data_generator.flow_from_directory('data/train',
                                                    target_size=(images_size,images_size),
                                                    batch_size=batch_size_training,
                                                    class_mode='categorical')

Found 30000 images belonging to 2 classes.


In [5]:
validation_generator = data_generator.flow_from_directory('data/valid',
                                                    target_size=(images_size,images_size),
                                                    batch_size=batch_size_validation,
                                                    class_mode='categorical')

Found 10000 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg',weights='imagenet'))
model.add(Dense(num_classes,activation='softmax'))

In [7]:
model.layers

In [8]:
model.layers[0].layers

In [9]:
# Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part,
#but to train only our dense output layer.
model.layers[0].trainable = False

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [13]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
300/300 [==============================] - 13994s 47s/step - loss: 0.0378 - accuracy: 0.9880 - val_loss: 0.1978 - val_accuracy: 0.9208
Epoch 2/2
300/300 [==============================] - 12582s 42s/step - loss: 0.0102 - accuracy: 0.9972 - val_loss: 0.1767 - val_accuracy: 0.9346


In [14]:
model.save('classifier_resnet_model.h5')